In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split
import pickle
from tqdm.notebook import tqdm 
import pyarrow as pa
import pyarrow.parquet as pq
import torch
from sklearn.metrics import confusion_matrix
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import sys
sys.path.append('..')
from utils.preprocess import NDF

2024-05-27 12:05:53.100735: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-27 12:05:53.100765: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-27 12:05:53.101710: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-27 12:05:53.696623: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


## Data Loading and Preprocessing

In [2]:
LEARNING_RATE = 0.001
BATCH_SIZE = 64
EPOCHS = 15


#The whole preprocessing step happens here
#Missing values set to -1
#Outliers are removed
#Encoding of categorical variables
#Min-Max scaling, sigmoid transformation (only if cnn is used)
input_data = {
    'benign': '../feature-extraction/floor/benign_combined.parquet', # benign_2312 + umbrella_benign_FINISHED
    #'malign': '../feature-extraction/floor/phishing_final_2024.parquet'
    'malign': '../feature-extraction/floor/malware_bp.parquet'
}

dataset = NDF("malware", True, input_data=input_data, one_line_processing=False)

# Convert the unique feature names to a list
feature_names_list = dataset['feature_names'].unique().tolist()

# Print each feature name individually
for feature in feature_names_list:
    print(feature)

print(dataset['feature_names'])

2024-05-27 12:05:56,186 - utils.preprocess - INFO - Benign dataset path: ../feature-extraction/floor/benign_combined.parquet
2024-05-27 12:05:56,187 - utils.preprocess - INFO - Malign dataset path: ../feature-extraction/floor/malware_bp.parquet
2024-05-27 12:05:56,735 - utils.preprocess - INFO - Number of records in combined dataset: 962074
2024-05-27 12:06:53,606 - utils.preprocess - INFO - Decision tree model saved to models/decision_tree_model.joblib
2024-05-27 12:06:54,720 - utils.preprocess - INFO - New feature 'dtree_prob' created from decision tree predictions.
2024-05-27 12:06:56,350 - utils.preprocess - INFO - Decision Tree Train Accuracy: 0.94
2024-05-27 12:06:56,351 - utils.preprocess - INFO - Decision Tree Test Accuracy: 0.92
2024-05-27 12:09:01,911 - utils.preprocess - INFO - Decision Tree Cross-Validation Scores: [0.91812705 0.91794594 0.91831389]
2024-05-27 12:09:02,150 - utils.preprocess - INFO - Generated class map: {'benign_2312:unknown': 0, 'benign_2310:unknown': 0, 


Dataset Subset:
Name: dataset_../feature-extraction/floor/malwarebp_2024-05-27.parquet
Features:
   Feature_0  Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  \
0   3.017475  -0.819933  -0.390788  -0.555299  -0.811427  -0.460858   
1  -0.331403  -0.089957  -0.390788   2.903392   1.653451   1.034639   
2  -0.331403  -0.819933  -0.390788  -0.555299  -0.811427  -0.460858   
3  -0.331403  -0.089957  -0.390788   0.136439  -0.811427   0.037641   
4   3.017475   0.640019   1.900324   0.828177   0.421012   0.037641   
5  -0.331403  -0.089957  -0.390788   0.828177   0.421012   0.037641   
6  -0.331403  -0.089957   1.900324  -0.555299  -0.811427   0.037641   
7  -0.331403  -0.819933  -0.390788  -0.555299  -0.811427  -0.460858   
8  -0.331403  -0.089957  -0.390788   0.136439   0.421012   0.037641   
9  -0.331403   2.099971  -0.390788   0.136439   1.653451   0.536140   

   Feature_6  Feature_7  Feature_8  Feature_9  ...  Feature_161  Feature_162  \
0  -0.954227  -0.486081  -0.274298  -0.2

## Smote Class Imbalance Handling

In [4]:
from collections import Counter
#imprt SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline


x_train, x_test, y_train, y_test = train_test_split(np.array(dataset['features']), np.array(dataset['labels']), test_size=0.2, random_state=42)

# Apply SMOTE
smote = SMOTE(random_state=42)
x_train_smote, y_train_smote = smote.fit_resample(x_train, y_train)

# Check the new class distribution
new_label_counts = Counter(y_train_smote)
print("New class distribution after SMOTE:")
for label, count in new_label_counts.items():
    print(f"Class {label}: {count}")

# Convert the data to PyTorch tensors
x_train = torch.tensor(x_train_smote, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)  # Convert x_test to tensor
y_train = torch.tensor(y_train_smote, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long) 
feature_size = x_train.shape[1]


New class distribution after SMOTE:
Class 0.0: 17427
Class 1.0: 17427


## Data Alignment

In [5]:
import torch.nn.functional as F
import math

# Function to calculate the next perfect square greater than a given number
def next_perfect_square(n):
    next_square = math.ceil(n**0.5)**2
    return next_square

feature_size = x_train.shape[1]
desired_size = next_perfect_square(feature_size)
side_size = int(desired_size ** 0.5)

# Calculate padding required to achieve the desired size
padding = desired_size - feature_size

# Applying dynamic padding
if padding > 0:
    x_train_padded = F.pad(x_train, (0, padding), 'constant', 0)
    x_test_padded = F.pad(x_test, (0, padding), 'constant', 0)
else:
    x_train_padded = x_train
    x_test_padded = x_test

# Reshape the data to the new dynamically calculated square shape
x_train = x_train_padded.view(-1, 1, side_size, side_size)
x_test = x_test_padded.view(-1, 1, side_size, side_size)

## CNN Architecture

In [6]:
class Net(nn.Module):
    def __init__(self, side_size):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        # Adjust the size calculation based on the number of convolutional layers
        self.fc1 = nn.Linear(128 * (side_size-6)**2, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, 2)
        
        # Optionally use dropout
        self.dropout1 = nn.Dropout(0.3)
        self.dropout2 = nn.Dropout(0.3)
        
        print("CNN model created")

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        
        x = torch.flatten(x, 1)
        x = self.dropout1(x)  # Dropout applied after flattening
        
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)  # Dropout applied after first fully connected layer
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.fc4(x) # No softmax here, as CrossEntropyLoss applies LogSoftmax internally

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Model's training

In [8]:
LEARNING_RATE = 0.000238
BATCH_SIZE = 64
EPOCHS = 15

model = Net(side_size).to(device)  # Classic fully connected model


#CANNOT RETURN SOFT MAX AS THE LOSS FUNCTION IS CROSS ENTROPY
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Prepare DataLoader
train_data = TensorDataset(x_train, y_train.long())
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

# Function to compute accuracy and F1 score
def compute_metrics(data_loader, model):
    model.eval()
    true_labels = []
    predictions = []

    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)  # Move data to the device
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            true_labels.extend(target.cpu().tolist())  # Move data back to CPU for scoring
            predictions.extend(pred.view_as(target).cpu().tolist())  # Move data back to CPU

    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    return accuracy, f1

epoch_losses = []
epoch_accuracies = []
epoch_f1s = []

# Training loop
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)  # Move data to the device
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Compute average loss
    avg_loss = running_loss / len(train_loader)
    epoch_losses.append(avg_loss)

    # Evaluate model and store metrics
    train_accuracy, train_f1 = compute_metrics(train_loader, model)
    epoch_accuracies.append(train_accuracy)
    epoch_f1s.append(train_f1)

    # Enhanced logging
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {avg_loss:.4f}, Accuracy: {train_accuracy:.4f}, F1 Score: {train_f1:.4f}")

# Save the model
torch.save(model.state_dict(), 'models/phishing_cnn_model_state_dict.pth')
print("Model training complete and saved.")

CNN model created
Epoch 1/3 - Loss: 0.1125, Accuracy: 0.9731, F1 Score: 0.9725
Epoch 2/3 - Loss: 0.0750, Accuracy: 0.9798, F1 Score: 0.9796
Epoch 3/3 - Loss: 0.0666, Accuracy: 0.9838, F1 Score: 0.9838
Model training complete and saved.


In [9]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import numpy as np
from sklearn.metrics import roc_auc_score

def compute_metrics(data_loader, model, criterion):
    """
    Computes a wide range of performance metrics for the given model evaluated on the provided DataLoader.
    
    Parameters:
        data_loader (DataLoader): DataLoader for evaluation data.
        model (torch.nn.Module): The neural network model to evaluate.
        criterion (torch.nn.Module): Loss function used for the model.
    
    Returns:
        dict: A dictionary containing various performance metrics.
    """
    model.eval()  # Switch model to evaluation mode
    true_labels, predictions, probs, losses = [], [], [], []

    with torch.no_grad():
        for data, target in data_loader:
            output = model(data)
            loss = criterion(output, target)
            pred = output.argmax(dim=1, keepdim=True)
            prob = torch.nn.functional.softmax(output, dim=1)[:,1]  # Probability for class 1
            true_labels.extend(target.cpu().numpy())
            predictions.extend(pred.cpu().view_as(target).numpy())
            probs.extend(prob.cpu().numpy())
            losses.append(loss.item())
    
    # Compute metrics
    metrics = {
        'accuracy': accuracy_score(true_labels, predictions),
        'f1': f1_score(true_labels, predictions, average='weighted'),
        'precision': precision_score(true_labels, predictions, average='weighted', zero_division=0),
        'recall': recall_score(true_labels, predictions, average='weighted', zero_division=0),
        'roc_auc_score': roc_auc_score(true_labels, probs) if len(np.unique(true_labels)) > 1 else 0,
        'cross_entropy_loss': np.mean(losses),
    }
    
    return metrics

def evaluate_model(model, x_test, y_test, batch_size, criterion):
    """
    Evaluates the model on the test dataset and prints out a comprehensive set of performance metrics.
    
    Parameters:
        model (torch.nn.Module): The neural network model to evaluate.
        x_test (Tensor): Test dataset features.
        y_test (Tensor): Test dataset labels.
        batch_size (int): Batch size for data loading.
        criterion (torch.nn.Module): Loss function used for the model.
    """
    test_data = TensorDataset(x_test, y_test.long())
    test_loader = DataLoader(test_data, batch_size=batch_size)
    
    metrics = compute_metrics(test_loader, model, criterion)
    
    # Display the metrics
    print("Test Metrics:")
    for metric_name, metric_value in metrics.items():
        if metric_name == 'confusion_matrix':
            print(f"{metric_name}:\n{metric_value}\n")
        else:
            print(f"{metric_name}: {metric_value:.4f}")
    
    return metrics

# Assuming criterion is defined (e.g., nn.CrossEntropyLoss())
criterion = torch.nn.CrossEntropyLoss()

# Example of how to use the evaluate_model function:
# model = Net()  # Assume Net is defined elsewhere and is your trained model
metrics = evaluate_model(model, x_test, y_test, BATCH_SIZE, criterion)

Test Metrics:
accuracy: 0.9793
f1: 0.9794
precision: 0.9795
recall: 0.9793
roc_auc_score: 0.9903
cross_entropy_loss: 0.0765
